In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('diabetes.csv')

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
x = df.iloc[:,:-1].values
y = df.iloc[:, -1].values

# .values returns a Numpy representation of the DataFrame.
# Only the values in the DataFrame will be returned, the axes labels
# will be removed.

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [8]:
x = scaler.fit_transform(x)

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=1)

In [57]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [11]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])


In [12]:
model.fit(x_train,y_train,batch_size=32, epochs=100, validation_data=(x_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 15ms/step - loss: 0.7200 - accuracy: 0.5212 - val_loss: 0.6748 - val_accuracy: 0.5519
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6551 - accuracy: 0.6303 - val_loss: 0.6217 - val_accuracy: 0.6948
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6107 - accuracy: 0.6726 - val_loss: 0.5837 - val_accuracy: 0.7403
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5799 - accuracy: 0.6938 - val_loss: 0.5573 - val_accuracy: 0.7857
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5580 - accuracy: 0.7166 - val_loss: 0.5375 - val_accuracy: 0.7857
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5403 - accuracy: 0.7296 - val_loss: 0.5222 - val_accuracy: 0.7857
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5260 - accuracy: 0.7459 - val_loss: 0.5097 - val_accuracy: 0.7922
Epoch 8/100


In [12]:
# 1.How to select approriate optimizer
# 2.No of nodes in a layer
# 3.How to select no of hidden layers
# 4.All in one model

In [13]:
pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.9 MB/s eta 0:00:00


In [14]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [15]:
# 1. How to select approriate optimizer
def build_model(hp):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [16]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)

In [17]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 08s


In [18]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [19]:
model = tuner.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model.fit(x_train,y_train,batch_size=32, epochs=100, initial_epoch=6, validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 14ms/step - loss: 0.5295 - accuracy: 0.7378 - val_loss: 0.5127 - val_accuracy: 0.7662
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5146 - accuracy: 0.7557 - val_loss: 0.5012 - val_accuracy: 0.7662
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5030 - accuracy: 0.7606 - val_loss: 0.4915 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4936 - accuracy: 0.7720 - val_loss: 0.4833 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4853 - accuracy: 0.7720 - val_loss: 0.4776 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4789 - accuracy: 0.7769 - val_loss: 0.4720 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4739 - accuracy: 0.7785 - val_loss: 0.4687 - val_accuracy: 0.7857
Epoch 14

In [22]:
# 2.No of nodes in a layer
# using the best optimizer we got above to find the best no of nodes
def build_model(hp):

  model = Sequential()
  units = hp.Int('units', min_value = 8, max_value = 128)
  # builds multiple models, in first model it takes 8 neurons, in second model it takes 16 neurons, then
  #  24 neurons and so on until 128 neurons, then it returns the neurons model that gave best performance
  model.add(Dense(units=units, activation='relu', input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [23]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',
                        max_trials=5,directory='mydir', project_name='ben')

In [24]:
tuner.search(x_train,y_train,epochs=5, validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 10s


In [25]:
tuner.get_best_hyperparameters()[0].values

{'units': 68}

In [26]:
model = tuner.get_best_models(num_models=1)[0]

In [27]:
model.fit(x_train,y_train,batch_size=32, epochs=100, initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 1s 4ms/step - loss: 0.5152 - accuracy: 0.7492
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4960 - accuracy: 0.7557
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4824 - accuracy: 0.7736
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4727 - accuracy: 0.7769
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4657 - accuracy: 0.7834
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4608 - accuracy: 0.7769
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4569 - accuracy: 0.7752
Epoch 14/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4541 - accuracy: 0.7801
Epoch 15/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4515 - accuracy: 0.7752
Epoch 16/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4490 - accuracy: 0.7785
Epo

In [31]:
# 3. nof of layers

def build_model(hp):
  model = Sequential()
  model.add(Dense(68, activation='relu', input_dim=8))
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(72,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [32]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=3, directory='mydir', project_name='num_layers')

Reloading Tuner from mydir/num_layers/tuner0.json


In [33]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 03m 04s


In [34]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7}

In [35]:
model = tuner.get_best_models(num_models=1)[0]

In [37]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6, validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 22ms/step - loss: 0.4574 - accuracy: 0.7801 - val_loss: 0.4801 - val_accuracy: 0.7922
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4550 - accuracy: 0.7801 - val_loss: 0.4790 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4507 - accuracy: 0.7834 - val_loss: 0.4774 - val_accuracy: 0.7922
Epoch 10/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4470 - accuracy: 0.7818 - val_loss: 0.4795 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4434 - accuracy: 0.7883 - val_loss: 0.4818 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4409 - accuracy: 0.7834 - val_loss: 0.4784 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4388 - accuracy: 0.7801 - val_loss: 0.4757 - val_accuracy: 0.7727
Epoch 14

In [61]:
# 4.no of layers and no of nodes in each layer
# we can also find activation, optimizer, loss fn

def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers', min_value=1, max_value = 10)):

    if counter == 0:

      model.add(Dense(hp.Int('units'+ str(i), min_value=8, max_value=128, step=8),activation= hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid']),input_dim=8))
      model.add(Dropout(hp.Choice('dropout' +str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:

      model.add(Dense(hp.Int('units'+ str(i), min_value=8, max_value=128, step=8),activation= hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid'])))
      model.add(Dropout(hp.Choice('dropout' +str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1

  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [62]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name='final1')

Reloading Tuner from mydir/final1/tuner0.json


In [63]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [64]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units0': 24,
 'activation0': 'relu',
 'dropout0': 0.5,
 'optimizer': 'adadelta',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1}

In [65]:
model = tuner.get_best_models(num_models=1)[0]

In [66]:
model.fit(x_train, y_train, epochs=200, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/200
20/20 [==============================] - 2s 17ms/step - loss: 0.6827 - accuracy: 0.6352 - val_loss: 0.6778 - val_accuracy: 0.6234
Epoch 8/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6814 - accuracy: 0.6189 - val_loss: 0.6778 - val_accuracy: 0.6234
Epoch 9/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6738 - accuracy: 0.6173 - val_loss: 0.6778 - val_accuracy: 0.6234
Epoch 10/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6823 - accuracy: 0.6238 - val_loss: 0.6778 - val_accuracy: 0.6234
Epoch 11/200
20/20 [==============================] - 0s 5ms/step - loss: 0.6758 - accuracy: 0.6107 - val_loss: 0.6777 - val_accuracy: 0.6234
Epoch 12/200
20/20 [==============================] - 0s 5ms/step - loss: 0.6745 - accuracy: 0.6254 - val_loss: 0.6777 - val_accuracy: 0.6234
Epoch 13/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6739 - accuracy: 0.6205 - val_loss: 0.6777 - val_accuracy: 0.6234
Epoch 14